In [ ]:
import os
import cv2
import dlib
import numpy as np
from PIL import Image, ImageFilter
import faceBlendCommon as faces

# Define directories
main_input_dir = 'D:/DU LIFE/Jan-june 2024/Masters Thesis/Real to Fake Generator/f/'
output_dir = 'D:/DU LIFE/Jan-june 2024/Masters Thesis/Real to Fake Generator/fake full/'
os.makedirs(output_dir, exist_ok=True)

# Load the landmark predictor
PREDICTOR_PATH = r"shape_predictor_68_face_landmarks.dat"
faceDetector = dlib.get_frontal_face_detector()
landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)

def createEyeMask(eyeLandmarks, im):
    eyeMask = np.zeros_like(im)
    cv2.fillConvexPoly(eyeMask, np.int32(eyeLandmarks), (255, 255, 255))
    return np.uint8(eyeMask)

def findIris(eyeMask, im, thresh):
    r = im[:,:,2]
    _, binaryIm = cv2.threshold(r, thresh, 255, cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4))
    morph = cv2.dilate(binaryIm, kernel, 1)
    morph = morph.astype(float) / 255
    eyeMask = eyeMask.astype(float) / 255
    return cv2.multiply(eyeMask, morph)

def findCentroid(iris):
    M = cv2.moments(iris[:,:,0])
    return (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])) if M["m00"] != 0 else (0, 0)

def createIrisMask(iris, centroid):
    cnts, _ = cv2.findContours(np.uint8(iris[:,:,0]), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return np.zeros_like(iris), np.ones_like(iris) * 255

    final_cnt = min(cnts, key=lambda cnt: cv2.pointPolygonTest(cnt, centroid, True))
    (x, y), radius = cv2.minEnclosingCircle(final_cnt)
    radius = max(int(radius) - 2, 0)

    irisMask = np.zeros_like(iris)
    inverseIrisMask = np.ones_like(iris) * 255
    if radius > 0:
        cv2.circle(irisMask, (int(x), int(y)), radius, (255, 255, 255), -1)
        cv2.circle(inverseIrisMask, (int(x), int(y)), radius, (0, 0, 0), -1)

    return cv2.GaussianBlur(irisMask, (5, 5), cv2.BORDER_DEFAULT), cv2.GaussianBlur(inverseIrisMask, (5, 5), cv2.BORDER_DEFAULT)

def changeEyeColor(im, irisMask, inverseIrisMask):
    imCopy = cv2.applyColorMap(im, cv2.COLORMAP_TWILIGHT_SHIFTED)
    return cv2.add(cv2.multiply(inverseIrisMask.astype(float) / 255, im.astype(float) / 255),
                   cv2.multiply(irisMask.astype(float) / 255, imCopy.astype(float) / 255))

def float642Uint8(im):
    return (255 * (im / np.amax(im))).astype(np.uint8)

# Iterate over subfolders and process images
for subfolder in os.listdir(main_input_dir):
    subfolder_path = os.path.join(main_input_dir, subfolder)
    if os.path.isdir(subfolder_path):
        for image_file in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_file)
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                print(f"Processing image: {image_path}")
                # Open the input image with PIL
                image = Image.open(image_path)
                base_name = os.path.splitext(image_file)[0]

                # Apply image filters
                filters = [
                    (ImageFilter.DETAIL, 'Face 1'),
                    (ImageFilter.EDGE_ENHANCE, 'Face 2'),
                    (ImageFilter.EDGE_ENHANCE_MORE, 'Face 3'),
                    (ImageFilter.SMOOTH, 'Face 4'),
                    (ImageFilter.SMOOTH_MORE, 'Face 5'),
                    (ImageFilter.SHARPEN, 'Face 6')
                ]

                for img_filter, suffix in filters:
                    filtered_image = image.filter(img_filter)
                    filtered_image.save(os.path.join(output_dir, f'{base_name} {suffix}.jpg'))
                    print(f"Saved filtered image: {base_name} {suffix}.jpg")

                # Convert the filtered image to OpenCV format for face landmark detection
                im = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
                landmarks = faces.getLandmarks(faceDetector, landmarkDetector, im)

                # Check if landmarks are detected
                if landmarks is not None and len(landmarks) > 0:
                    print("Landmarks detected.")
                    leftEyeMask = createEyeMask(landmarks[36:42], im)
                    rightEyeMask = createEyeMask(landmarks[43:49], im)
                    leftIris = findIris(leftEyeMask, im, 40)
                    rightIris = findIris(rightEyeMask, im, 50)

                    leftIrisCentroid = findCentroid(leftIris)
                    rightIrisCentroid = findCentroid(rightIris)

                    leftIrisMask, leftInverseIrisMask = createIrisMask(leftIris, leftIrisCentroid)
                    rightIrisMask, rightInverseIrisMask = createIrisMask(rightIris, rightIrisCentroid)

                    coloredEyesLady = changeEyeColor(im, rightIrisMask, rightInverseIrisMask)
                    coloredEyesLady = float642Uint8(coloredEyesLady)
                    coloredEyesLady = changeEyeColor(coloredEyesLady, leftIrisMask, leftInverseIrisMask)

                    # Save the modified image
                    output_image_path = os.path.join(output_dir, f'{base_name} Colored Eyes.jpg')
                    cv2.imwrite(output_image_path, coloredEyesLady)
                    print(f"Saved modified image with colored eyes: {output_image_path}")
                else:
                    print("No landmarks detected for:", image_file)

# Indicate completion
print(f"Processed images saved in: {output_dir}")
